In [442]:
import pandas as pd
import openpyxl
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.stats import describe, ttest_1samp
from pathlib import Path
from openpyxl import load_workbook
import os

In [443]:

#get the sorted fin file
# file_name_str = r"..\Refinitiv\Analysis\AU\AU_Refinitiv_fin_data_portfolio.xlsx"
# file_name_str = r"..\Refinitiv\Analysis\CN\CN_Refinitiv_fin_data_portfolio.xlsx"
# file_name_str = r"..\Refinitiv\Analysis\HK\HK_Refinitiv_fin_data_portfolio.xlsx"
# file_name_str = r"..\Refinitiv\Analysis\SG\SG_Refinitiv_fin_data_portfolio.xlsx"
file_name_str = r"..\Refinitiv\Analysis\MY\MY_Refinitiv_fin_data_portfolio.xlsx"

# indicator = 'e'
# indicator = 's'
# indicator = 'g'
# indicator = 'esg'
# indicator = 'esg_value'
# indicator = 'value'
# indicator = 'mv'
# indicator = 'roa'
indicator = 'vol_180d'
## indicator = 'return_com_eqy'


file_name = Path(file_name_str).stem

lo_quantile_folder = "..\..\Refinitiv\Analysis_bottom\\" + file_name[:2] + "\\" + indicator
lo_reporting_path = lo_quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting_low.xlsx"
lo_fin_file = pd.ExcelFile(lo_reporting_path)
lo_findata_sheet = pd.read_excel(
    lo_fin_file, sheet_name="main", header=0)

lo_porfolio_selection = pd.DataFrame(lo_findata_sheet)

hi_quantile_folder = "..\..\Refinitiv\Analysis_top\\" + file_name[:2] + "\\" + indicator
hi_reporting_path = hi_quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting_second_highest.xlsx"
hi_fin_file = pd.ExcelFile(hi_reporting_path)
hi_findata_sheet = pd.read_excel(
    hi_fin_file, sheet_name="main", header=0)

hi_porfolio_selection = pd.DataFrame(hi_findata_sheet)


In [444]:
lo_porfolio_selection.columns

Index(['year', 'no of firms', 'no of firms(bottom 20)', 'rm_r_y1', 'rm_r_y2',
       'eq_wg_ret_p', 'fc_wg_ret_p', 'lo_eq_wg_r_p', 'lo_eq_wg_r_p_y1',
       'eq_r_alpha_y1', 'lo_eq_wg_r_p_y2', 'eq_r_alpha_y2', 'lo_fc_wg_r_p',
       'lo_fc_wg_r_p_y1', 'fc_r_alpha_y1', 'lo_fc_wg_r_p_y2', 'fc_r_alpha_y2',
       'lo_eq_wg_rc_p', 'lo_eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2',
       'eq_rc_alpha_y2', 'lo_fc_wg_rc_p', 'lo_fc_wg_rc_p_y1', 'fc_rc_alpha_y1',
       'lo_fc_wg_rc_p_y2', 'fc_rc_alpha_y2'],
      dtype='object')

In [445]:
hi_porfolio_selection.columns

Index(['year', 'no of firms', 'no of firms(top 20)', 'rm_r_y1', 'rm_r_y2',
       'eq_wg_ret_p', 'fc_wg_ret_p', 'eq_wg_r_p', 'eq_wg_r_p_y1',
       'eq_r_alpha_y1', 'eq_wg_r_p_y2', 'eq_r_alpha_y2', 'fc_wg_r_p',
       'fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_wg_r_p_y2', 'fc_r_alpha_y2',
       'eq_wg_rc_p', 'eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2',
       'eq_rc_alpha_y2', 'fc_wg_rc_p', 'fc_wg_rc_p_y1', 'fc_rc_alpha_y1',
       'fc_wg_rc_p_y2', 'fc_rc_alpha_y2'],
      dtype='object')

In [446]:
if indicator == 'historical_market_cap' or indicator == 'vol_180d':
    hi_porfolio_selection['eq_r_alpha3_y1'] =  lo_porfolio_selection['lo_eq_wg_r_p_y1']- \
    hi_porfolio_selection['eq_wg_r_p_y1']

    hi_porfolio_selection['fc_r_alpha3_y1'] = lo_porfolio_selection['lo_fc_wg_r_p_y1'] - \
        hi_porfolio_selection['fc_wg_r_p_y1']

    hi_porfolio_selection['eq_rc_alpha3_y1'] =  lo_porfolio_selection['lo_eq_wg_rc_p_y1']- \
        hi_porfolio_selection['eq_wg_rc_p_y1']

    hi_porfolio_selection['fc_rc_alpha3_y1'] =  lo_porfolio_selection['lo_fc_wg_rc_p_y1'] - \
        hi_porfolio_selection['fc_wg_rc_p_y1']

else:
    hi_porfolio_selection['eq_r_alpha3_y1'] = hi_porfolio_selection['eq_wg_r_p_y1'] - \
        lo_porfolio_selection['lo_eq_wg_r_p_y1']

    hi_porfolio_selection['fc_r_alpha3_y1'] = hi_porfolio_selection['fc_wg_r_p_y1'] - \
        lo_porfolio_selection['lo_fc_wg_r_p_y1']

    hi_porfolio_selection['eq_rc_alpha3_y1'] = hi_porfolio_selection['eq_wg_rc_p_y1'] - \
        lo_porfolio_selection['lo_eq_wg_rc_p_y1']

    hi_porfolio_selection['fc_rc_alpha3_y1'] = hi_porfolio_selection['fc_wg_rc_p_y1'] - \
        lo_porfolio_selection['lo_fc_wg_rc_p_y1']

hi_porfolio_selection.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
4,2021,331.000000,82.000000,-0.016569,-0.016443,-0.055119,-0.054960,-0.070467,-0.845598,-0.829030,...,0.198269,0.014015,0.168185,0.184753,0.181774,0.198218,-2.089240,-2.188189,0.009183,0.008792
5,2022,267.000000,66.000000,NaN,NaN,0.024679,0.031052,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,average,320.333333,79.333333,-0.186360,0.095896,-0.008647,-0.008015,-0.030515,-0.366178,-0.179818,...,0.378314,-0.005857,-0.070285,0.116075,0.475770,0.379875,-0.572337,-0.619539,0.103294,0.098893
7,t-score,NaN,NaN,-0.450671,0.259597,-0.175080,-0.180843,-0.354998,-0.354998,-0.127133,...,1.699208,-0.125963,-0.125963,0.801768,0.803721,1.696081,-1.216469,-1.231124,0.227542,0.210447
8,p-value,NaN,NaN,0.675575,0.807988,0.869522,0.865284,0.740518,0.740518,0.904970,...,0.164507,0.905839,0.905839,0.467613,0.466605,0.165112,-0.549347,-0.549957,0.017187,0.030892


In [447]:
portfolio_selection_variances_t_test = hi_porfolio_selection.copy()
portfolio_selection_variances_t_test = hi_porfolio_selection.iloc[:-3]
portfolio_selection_variances_t_test.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
1,2018,331.0,82.0,-1.365354,-0.765306,0.216854,0.221937,0.199011,2.388137,3.753491,...,-0.068612,-0.139255,-1.671061,-0.305707,-0.834616,-0.069310,-2.116689,-2.111661,0.756775,0.724532
2,2019,331.0,82.0,-0.894749,-0.605392,0.094109,0.088765,0.076285,0.915416,1.810165,...,-0.049880,-0.085124,-1.021486,-0.126737,-0.656167,-0.050774,-0.959465,-0.947318,0.495933,0.474803
3,2020,331.0,82.0,0.562436,0.732651,-0.024815,-0.029173,-0.040551,-0.486611,-1.049046,...,0.807950,0.080923,0.971075,0.408639,1.544115,0.811463,0.156712,0.166262,-0.311741,-0.298459
4,2021,331.0,82.0,-0.016569,-0.016443,-0.055119,-0.054960,-0.070467,-0.845598,-0.829030,...,0.198269,0.014015,0.168185,0.184753,0.181774,0.198218,-2.089240,-2.188189,0.009183,0.008792
5,2022,267.0,66.0,NaN,NaN,0.024679,0.031052,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [448]:
SG_esg_int_mean = portfolio_selection_variances_t_test.mean(numeric_only=True)
SG_esg_int_mean = pd.DataFrame(SG_esg_int_mean).T
SG_esg_int_mean.insert(0, 'year', 'Average', True)
SG_esg_int_mean = pd.concat(
    [portfolio_selection_variances_t_test, SG_esg_int_mean], axis=0)
SG_esg_int_mean.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
2,2019,331.000000,82.000000,-0.894749,-0.605392,0.094109,0.088765,0.076285,0.915416,1.810165,...,-0.049880,-0.085124,-1.021486,-0.126737,-0.656167,-0.050774,-0.959465,-0.947318,0.495933,0.474803
3,2020,331.000000,82.000000,0.562436,0.732651,-0.024815,-0.029173,-0.040551,-0.486611,-1.049046,...,0.807950,0.080923,0.971075,0.408639,1.544115,0.811463,0.156712,0.166262,-0.311741,-0.298459
4,2021,331.000000,82.000000,-0.016569,-0.016443,-0.055119,-0.054960,-0.070467,-0.845598,-0.829030,...,0.198269,0.014015,0.168185,0.184753,0.181774,0.198218,-2.089240,-2.188189,0.009183,0.008792
5,2022,267.000000,66.000000,NaN,NaN,0.024679,0.031052,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Average,320.333333,79.333333,-0.186360,0.095896,-0.008647,-0.008015,-0.030515,-0.366178,-0.179818,...,0.378314,-0.005857,-0.070285,0.116075,0.475770,0.379875,-0.572337,-0.619539,0.103294,0.098893


In [449]:
SG_res_dict_t_test = {}
temp_list = []

"""
make the t-stat, p-value into dictionary
"""
for column in range(3, portfolio_selection_variances_t_test.columns.__len__()):
    tscore, pvalue = ttest_1samp(
        a=portfolio_selection_variances_t_test.dropna()[portfolio_selection_variances_t_test.columns[column]], popmean=0)
    temp_dict = {portfolio_selection_variances_t_test.columns[column]: {
        "t-score": tscore, "p-value": pvalue}}
    SG_res_dict_t_test = {**SG_res_dict_t_test, **temp_dict}

"""
make the dictionary into dataframe
"""
res_df = pd.DataFrame.from_dict({(i): SG_res_dict_t_test[i]
                                 for i in SG_res_dict_t_test.keys()},
                                orient='index').T
res_df.reset_index(inplace=True)
res_df.rename(columns={'index': 'year'}, inplace=True)
res_df
# SG_res_dict = {**temp_dict, **SG_res_dict}
# testing_df = pd.DataFrame(SG_res_dict, index=[0])
# testing_df
# SG_res_dict


,year,rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,eq_wg_r_p_y2,eq_r_alpha_y2,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
0,t-score,-0.450671,0.259597,-0.175080,-0.180843,-0.354998,-0.354998,-0.127133,0.884680,0.703304,...,1.699208,-0.125963,-0.125963,0.801768,0.803721,1.696081,-0.716480,-0.793683,0.450671,0.450671
1,p-value,0.675575,0.807988,0.869522,0.865284,0.740518,0.740518,0.904970,0.426304,0.520646,...,0.164507,0.905839,0.905839,0.467613,0.466605,0.165112,0.513301,0.471805,0.675575,0.675575


In [450]:
SG_esg_int = pd.concat([SG_esg_int_mean, res_df], axis=0)
SG_esg_int.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
4,2021,331.000000,82.000000,-0.016569,-0.016443,-0.055119,-0.054960,-0.070467,-0.845598,-0.829030,...,0.198269,0.014015,0.168185,0.184753,0.181774,0.198218,-2.089240,-2.188189,0.009183,0.008792
5,2022,267.000000,66.000000,NaN,NaN,0.024679,0.031052,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Average,320.333333,79.333333,-0.186360,0.095896,-0.008647,-0.008015,-0.030515,-0.366178,-0.179818,...,0.378314,-0.005857,-0.070285,0.116075,0.475770,0.379875,-0.572337,-0.619539,0.103294,0.098893
0,t-score,NaN,NaN,-0.450671,0.259597,-0.175080,-0.180843,-0.354998,-0.354998,-0.127133,...,1.699208,-0.125963,-0.125963,0.801768,0.803721,1.696081,-0.716480,-0.793683,0.450671,0.450671
1,p-value,NaN,NaN,0.675575,0.807988,0.869522,0.865284,0.740518,0.740518,0.904970,...,0.164507,0.905839,0.905839,0.467613,0.466605,0.165112,0.513301,0.471805,0.675575,0.675575


In [451]:
export_file_path = "..\..\Refinitiv\Analysis\\" + file_name[:2] + "\\" + indicator + "\\" + file_name[:2]+ '_' + indicator + "_alpha3_int_reporting.xlsx"
SG_esg_int.to_excel(export_file_path , sheet_name="main", index=False)